In [57]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.api as sm

#from skbio.stats.composition import clr
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer

# unumHelpers.py should be in the same folder as this ipynb file
#from unumHelpers import unumOLS


def color_big_red(val):
    color = 'red' if abs(val) >0.4 else 'black'
    return 'color: %s' % color

In [58]:
data = pd.read_excel('2020_CO_terms_cleaned.xlsx')

In [59]:
pca_cats = pd.read_excel('2020 Dems Category Mapping.xlsx')
dependents = pd.read_excel('2020 Dependent.xlsx')

In [60]:
dependents = dependents.rename(columns = {'Unnamed: 0':'Date'})

## Top_20 Campaign Finance 

In [66]:
top_20 = ['Date','lobby','lobbying', 'contribution', 'lobbyist', 'sponsor', 'pac', 
          'citizens united', 'contributions', 'contribute', 'pacs', 'career',
          'lobbyists','fundraising','fundraiser','contributing','koch','contributed',
          'campaign finance','sponsors','loophole']

In [73]:
reg_df = pd.merge(data,pca_cats,on='Category',how='left').groupby('Term').sum().T
reg_df = reg_df.drop('Unnamed: 0', axis = 0)
reg_df = reg_df.reset_index().rename(columns={'index':'Date'})
reg_df['Date'] = pd.to_datetime(reg_df['Date'], infer_datetime_format=True)
reg_df = reg_df.filter(top_20)
reg_df = pd.merge(reg_df,dependents,on='Date',how='inner')
reg_df = reg_df.fillna(0)

In [74]:
reg_df.head()

,Date,lobby,lobbying,lobbyist,sponsor,pac,citizens united,contributions,contribute,pacs,...,contributing,koch,contributed,campaign finance,sponsors,loophole,Hickenlooper_raw,Gardner_raw,Hickenlooper_adj,Gardner_adj
0,2019-02-15,0.000135,0.000092,0.000052,0.000106,0.000055,0.000025,0.000083,0.000037,0.000029,...,0.000016,0.000024,0.000010,0.000010,0.000011,0.000008,0.431907,0.568093,0.474177,0.525823
1,2019-02-16,0.000125,0.000043,0.000065,0.000061,0.000023,0.000000,0.000026,0.000023,0.000029,...,0.000023,0.000032,0.000016,0.000013,0.000023,0.000000,0.431490,0.568510,0.474032,0.525968
2,2019-02-17,0.000155,0.000061,0.000088,0.000164,0.000077,0.000051,0.000036,0.000040,0.000034,...,0.000036,0.000000,0.000016,0.000000,0.000039,0.000034,0.430931,0.569069,0.473836,0.526164
3,2019-02-18,0.000155,0.000029,0.000019,0.000092,0.000029,0.000033,0.000059,0.000026,0.000021,...,0.000023,0.000029,0.000023,0.000014,0.000025,0.000022,0.430737,0.569263,0.473768,0.526232
4,2019-02-19,0.000161,0.000089,0.000093,0.000107,0.000054,0.000025,0.000077,0.000049,0.000025,...,0.000015,0.000025,0.000023,0.000010,0.000021,0.000009,0.432308,0.567692,0.474318,0.525682


In [75]:
y = reg_df[['Gardner_adj','Hickenlooper_adj']]
reg_df.corr()[['Gardner_adj','Hickenlooper_adj']]

,Gardner_adj,Hickenlooper_adj
lobby,0.018025,-0.018025
lobbying,0.290388,-0.290388
lobbyist,0.208120,-0.208120
sponsor,-0.106903,0.106903
pac,0.264662,-0.264662
citizens united,0.126104,-0.126104
contributions,0.315030,-0.315030
contribute,0.228835,-0.228835
pacs,0.109844,-0.109844
career,0.067385,-0.067385
